In [0]:
import pandas as pd
import numpy as np

In [2]:
#loading data
data=pd.read_csv("Tweets.csv")
data.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [3]:
#splitting data into training and testing
from sklearn.model_selection import train_test_split
train,test=train_test_split(data)
train.shape,test.shape

((10980, 15), (3660, 15))

In [0]:
#forming training and testing data
xtrain=train["text"]
ytrain=train["airline_sentiment"]
xtest=test["text"]
ytest=test["airline_sentiment"]

In [5]:
#number of negative,neutral & positive tweets in training data
neg=0
neu=0
pos=0
for sent in ytrain:
    if sent=="negative":
        neg+=1
    elif sent=="neutral":
        neu+=1
    else:
        pos+=1
neg,neu,pos
print("number of negative,neutral & positive tweets in training data: ",(neg,neu,pos))
neg=0
neu=0
pos=0
for sent in ytest:
    if sent=="negative":
        neg+=1
    elif sent=="neutral":
        neu+=1
    else:
        pos+=1
neg,neu,pos
print("number of negative,neutral & positive tweets in training data: ",(neg,neu,pos))

number of negative,neutral & positive tweets in training data:  (6910, 2311, 1759)
number of negative,neutral & positive tweets in training data:  (2268, 788, 604)


In [6]:
#downloading dependencies
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [0]:
#forming set of stopwords
from nltk.corpus import stopwords
stop=set(stopwords.words("english"))
import string
punctuations=list(string.punctuation)
stop.update(punctuations)

In [0]:
#creating lemmatizer
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.corpus import wordnet
Lemmatizer=WordNetLemmatizer()

In [0]:
#function for converting tags into required form
def simple_pos(tag):
    if tag.startswith("N"):
        return wordnet.NOUN
    elif tag.startswith("V"):
        return wordnet.VERB
    elif tag.startswith("J"):
        return wordnet.ADJ
    elif tag.startswith("R"):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [10]:
#sample of training data
xtrain[5]

"@VirginAmerica seriously would pay $30 a flight for seats that didn't have this playing.\nit's really the only bad thing about flying VA"

In [0]:
#tokenizing
from nltk import word_tokenize
x_train=[]
for text in xtrain:
    x_train.append(word_tokenize(text))
x_test=[]
for text in xtest:
    x_test.append(word_tokenize(text))

In [12]:
#sample after tokenization
x_train[5]

['@',
 'JetBlue',
 '@',
 'pilyoc',
 'dont',
 'talk',
 'about',
 'my',
 'friend',
 '@',
 'JetBlue',
 'like',
 'that',
 '.',
 '#',
 'thefutureisweird']

In [0]:
#function to clean data i.e. to apply lemmatization
def clean(text_data):
    X_Train=[]
    for text in text_data:
        clean_words=[]
        for word in text:
            if word.lower() not in stop:
                tag=pos_tag([word])[0][1]
                clean_word=Lemmatizer.lemmatize(word,simple_pos(tag))
                clean_words.append(clean_word)
        X_Train.append(clean_words)
    return X_Train

In [0]:
X_Train=clean(x_train)
X_Test=clean(x_test)

In [15]:
#sample after cleaning
X_Train[5]

['JetBlue',
 'pilyoc',
 'dont',
 'talk',
 'friend',
 'JetBlue',
 'like',
 'thefutureisweird']

In [0]:
#joining the words in list to form a string
xTrain=[" ".join(text) for text in X_Train]
xTest=[" ".join(text) for text in X_Test]

In [0]:
#applying count vectorizer to data
from sklearn.feature_extraction.text import CountVectorizer
count_vec=CountVectorizer(max_features=3000,ngram_range=(1,3),max_df=0.8)
Xtrain=count_vec.fit_transform(xTrain)
Xtest=count_vec.transform(xTest)

In [18]:
#applying Multinomial Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf=MultinomialNB()
clf.fit(Xtrain,ytrain)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [19]:
#score of Multinomial Naive Bayes
train_score=clf.score(Xtrain,ytrain)
test_score=clf.score(Xtest,ytest)
print("Training Accuracy of Multinomial Naive Bayes: ",train_score)
print("Testing Accuracy of Multinomial Naive Bayes: ",test_score)

Training Accuracy of Multinomial Naive Bayes:  0.8087431693989071
Testing Accuracy of Multinomial Naive Bayes:  0.7718579234972678


In [20]:
#applying Support Vector Machine
from sklearn.svm import SVC
svc=SVC(C=55,gamma=0.0025)
svc.fit(Xtrain,ytrain)
train_score=svc.score(Xtrain,ytrain)
test_score=svc.score(Xtest,ytest)
print("Training Accuracy of Support Vetor Machine (SVC): ",train_score)
print("Testing Accuracy of Support Vetor Machine (SVC): ",test_score)

Training Accuracy of Support Vetor Machine (SVC):  0.8794171220400728
Testing Accuracy of Support Vetor Machine (SVC):  0.7879781420765027


In [0]:
#predicting using SVC classifier
Y_train_pred=svc.predict(Xtrain)
Y_test_pred=svc.predict(Xtest)

In [22]:
#printing training results of SVC
from sklearn.metrics import classification_report,confusion_matrix
print("Training classification report of SCV")
print(classification_report(ytrain,Y_train_pred))
print("Training confusion matrix of SCV")
print(confusion_matrix(ytrain,Y_train_pred))

Training classification report of SCV
              precision    recall  f1-score   support

    negative       0.91      0.95      0.93      6910
     neutral       0.79      0.74      0.76      2311
    positive       0.88      0.79      0.83      1759

   micro avg       0.88      0.88      0.88     10980
   macro avg       0.86      0.83      0.84     10980
weighted avg       0.88      0.88      0.88     10980

Training confusion matrix of SCV
[[6545  291   74]
 [ 472 1717  122]
 [ 195  170 1394]]


In [23]:
#printing testing results of SVC
print("Testing classification report of SCV")
print(classification_report(ytest,Y_test_pred))
print("Testing confusion matrix of SCV")
print(confusion_matrix(ytest,Y_test_pred))

Testing classification report of SCV
              precision    recall  f1-score   support

    negative       0.84      0.88      0.86      2268
     neutral       0.62      0.61      0.61       788
    positive       0.79      0.66      0.72       604

   micro avg       0.79      0.79      0.79      3660
   macro avg       0.75      0.72      0.73      3660
weighted avg       0.79      0.79      0.79      3660

Testing confusion matrix of SCV
[[2007  201   60]
 [ 260  480   48]
 [ 111   96  397]]
